In [4]:
import pandas as pd
import seaborn as sb
import numpy as np


In [95]:
import os


def get_tuneBooks_file_names(path):
    tuneBook_filenames = []
    
    for tuneBook_name in os.listdir(path):
        if tuneBook_name.endswith('.abc'):
            tuneBook_filenames.append(tuneBook_name)
    
    return ["xmas.abc"]
    #return tuneBook_filenames
def split_abc_song(abc_song):
    k_index = abc_song.find('K:')

    split_index = abc_song.find('\n', k_index)

    header = abc_song[:split_index]
    body = abc_song[split_index+1:]
    return header, body

def tuneBook_to_dataframe(tuneBook,tuneBook_name):
    song_list = contents.split("\n\n")
    song_list = [song.strip() for song in song_list]
    songs_header_body_format = [split_abc_song(song) for song in song_list]
    
    #for (h,b) in songs_header_body_format:
    #    print("header")
    #    print(h)
    #    print("*"*10)
    #    print("body")
    #    print(b)
     #   print("\n")
    #    print("-"*50)
     #   print("\n")
    return pd.DataFrame(songs_header_body_format, columns=['header', 'body'])



absolute_path = os.getcwd()
relative_path = "notebooks/data/NottinghamData/nottingham_database"
absolute_path = os.path.join(absolute_path, relative_path)

list_tuneBooks = get_tuneBooks_file_names(absolute_path)
for abc_tuneBook_filename in list_tuneBooks:
    file_path = os.path.join(absolute_path, abc_tuneBook_filename)
    
    with open(file_path) as tuneBook:
        contents = tuneBook.read()
        df = tuneBook_to_dataframe(contents,abc_tuneBook_filename)

df.T         

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
header,X: 1\nT:The Boar's Head\n% Nottingham Music Da...,X: 2\nT:Ding Dong\n% Nottingham Music Database...,X: 3\nT:GKW\n% Nottingham Music Database\n%P:A...,"X: 4\nT:God rest you, merry gentlemen\n% Notti...",X: 5\nT:Good Christian Men Rejoice\n% Nottingh...,X: 6\nT:The Holly and the Ivy\n% Nottingham Mu...,X: 7\nT:Jubilate Deo (round)\n% Nottingham Mus...,X: 8\nT:Nos Galan\n% Nottingham Music Database...,% continue here,X: 9\nT:On Christmas Night\n% Nottingham Music...,X: 10\nT:Wassail 1\n% Nottingham Music Databas...,X: 11\nT:Wassail 2\n% Nottingham Music Databas...,X: 12\nT:Wassail 3\n% Nottingham Music Databas...,X: 13\nT:W3KOOA\n% Nottingham Music Database\n...,
body,"|:G|""C""c2 cc|""G""Bc ""C""G3/2E/2|""F""FF ""Dm""AF| [1...","P:A\n""Bb""BB ""Eb""c/2B/2A/2G/2|""F""F3F|""Eb""GB ""F7...","%P:A\n""G""GG G""D""A|""G""G""C""G ""D""D2|""C""E""G""D ""C""E...","E|""Em""EB BA|""Em""GF ED|""Em""EF GA|""B7""B3E|""Em""EB...","F|""F""F2F A2B|""F""c2d ""C7""c2c|""F""F2F A2B|""F""c2d ...","K:F\n=F\\nP:And\nd\\nP:the\ne|\\nP:ri-\n""D""fe\...","""C""c2 ""G7""d2|ec cB|cA GF|EF D2|C2 G,G,|C2 z2||","P:A\n|:""F""c3/2B/2 AG|""F""FG AF|""Bb""G/2A/2B/2G/2...",% continue here %,"d|""G""d2B ""C""c2d|""G""BAG ""D7""A2F|""G""G2G ""C""ABc|""...","D|""Dm""D2A A2G|""Dm""F2F F2E|""Dm""D2E F2G|""A7""A3 -...","D|""D""D2A A2B|""D""A2F D2C|""D""D2E F2G|""A7""A3 -A2D...","G|""G""G2A B2A|""G""G2A B2A|""G""G2d d2d|""D7""d3 -d2:...","""Em""B2A G2E|""B7""FGF ""Em""E3|""Em""B2A G2E|""B7""FGF...",
